In [1]:
!pip install pytorch_tabnet

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as torch_optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

In [3]:
path = './drive/MyDrive/Materiale_Pellegrino_personal/NSLKDD/NSLKDD_Full.csv'
dataset = pd.read_csv(path)

In [4]:
dataset

,duration,protocol_type,service,flag,symbolic,src_bytes,dst_bytes,DoS,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,binlabel,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,20,normal,normal
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,15,normal,normal
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,19,anomaly,DoS
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,21,normal,normal
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,21,normal,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148511,0,tcp,smtp,SF,794,333,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.00,0.00,0.00,100,141,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,21,normal,normal
148512,0,tcp,http,SF,317,938,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,11,0.0,0.0,0.0,0.0,1.00,0.00,0.18,197,255,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,21,normal,normal
148513,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,10,0.0,0.0,0.0,0.0,1.00,0.00,0.20,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,15,anomaly,DoS
148514,0,udp,domain_u,SF,42,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,0.0,0.0,0.0,0.0,1.00,0.00,0.33,255,252,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,21,normal,normal


In [5]:
# Elimino la colonna 'num_access_files' poiché inutile
dataset = dataset.drop('num_access_files', axis=1)

# Elimino la colonna 'binlabel' dal dataset
dataset = dataset.drop('binlabel', axis=1)

In [6]:
dep_var = 'label'
cat_names = ["protocol_type", "service", "flag"]
cont_names = [col for col in dataset.columns if col not in cat_names and col != dep_var]

In [7]:
# LabelEncoding della variabile target 
target_index = dataset.columns.get_loc(dep_var)
dataset.iloc[:, target_index] = LabelEncoder().fit_transform(dataset[dep_var])

#LabelEncoding delle variabili categoriali
for col in cat_names:
  target_index = dataset.columns.get_loc(col)
  dataset.iloc[:, target_index] = LabelEncoder().fit_transform(dataset[col])

In [8]:
features = [col for col in dataset.columns if col != dep_var]

cat_idxs = [i for i, f in enumerate(features) if f in cat_names]

cat_dims = []
for col in cat_names:
  cat_dims.append(len(Counter(dataset[col])))

In [9]:
print(features)
print(cat_idxs)
print(cat_dims)

['duration', 'protocol_type', 'service', 'flag', 'symbolic', 'src_bytes', 'dst_bytes', 'DoS', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']
[1, 2, 3]
[3, 70, 11]


In [10]:
from sklearn.model_selection import train_test_split

# train 50% e test 50%
train, test = train_test_split(dataset, test_size=0.50)

In [11]:
y_train = train[dep_var]
train = train.drop(dep_var, axis=1)
y_test = test[dep_var]
test = test.drop(dep_var, axis=1)

# validation di 2500 righe da train
train, validation, y_train, y_val = train_test_split(train, y_train, test_size=0.033666, random_state=0)

In [12]:
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

y_train = np.array(y_train)
y_val = np.array(y_val)

In [13]:
import pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

In [14]:
model = TabNetClassifier(n_d=64, n_a=64, n_steps=5, optimizer_fn=torch.optim.Adam, cat_idxs=cat_idxs, cat_dims=cat_dims,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":10, # how to use learning rate scheduler
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax"
                      )

Device used : cuda


In [15]:
# fit the model 
model.fit(
    train,y_train,
    eval_set=[(train, y_train), (validation, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=500 , patience=50,
    batch_size=512, virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
)  

epoch 0  | loss: 0.26518 | train_accuracy: 0.91328 | valid_accuracy: 0.8996  |  0:00:12s
epoch 1  | loss: 0.07448 | train_accuracy: 0.98429 | valid_accuracy: 0.9828  |  0:00:25s
epoch 2  | loss: 0.05654 | train_accuracy: 0.98509 | valid_accuracy: 0.9812  |  0:00:38s
epoch 3  | loss: 0.04041 | train_accuracy: 0.98859 | valid_accuracy: 0.9872  |  0:00:51s
epoch 4  | loss: 0.03494 | train_accuracy: 0.99058 | valid_accuracy: 0.9888  |  0:01:04s
epoch 5  | loss: 0.03412 | train_accuracy: 0.99188 | valid_accuracy: 0.9912  |  0:01:16s
epoch 6  | loss: 0.02847 | train_accuracy: 0.9892  | valid_accuracy: 0.986   |  0:01:29s
epoch 7  | loss: 0.03409 | train_accuracy: 0.99137 | valid_accuracy: 0.9904  |  0:01:42s
epoch 8  | loss: 0.03166 | train_accuracy: 0.99352 | valid_accuracy: 0.9924  |  0:01:54s
epoch 9  | loss: 0.02493 | train_accuracy: 0.99271 | valid_accuracy: 0.9904  |  0:02:07s
epoch 10 | loss: 0.02136 | train_accuracy: 0.99395 | valid_accuracy: 0.9908  |  0:02:19s
epoch 11 | loss: 0.02

In [16]:
y_pred = model.predict(test)

In [17]:
print('Test:', Counter(y_test))
print('Pred:', Counter(y_pred))

Test: Counter({2: 38565, 0: 26434, 1: 7141, 3: 2055, 4: 63})
Pred: Counter({2: 38430, 0: 26445, 1: 7172, 3: 2119, 4: 92})


In [18]:
# Matrice di confusione, accuracy, classification_report
from sklearn.metrics import *

# y_test è la variabile che contiene i valori effettivi
# y_pred contiene i valori predetti dal modello
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

acc = accuracy_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
# non presente nella libreria, calcolo mediante formula
f2 = (1+2**2)*((precision*recall)/((2**2*precision)+recall))

In [19]:
mcm = multilabel_confusion_matrix(y_test, y_pred)
print(mcm)

[[[47795    29]
  [   18 26416]]

 [[67066    51]
  [   20  7121]]

 [[35672    21]
  [  156 38409]]

 [[72111    92]
  [   28  2027]]

 [[74149    46]
  [   17    46]]]


In [20]:
print(cm)

[[26416    11     3     1     3]
 [    0  7121    10     8     2]
 [   28    36 38409    70    22]
 [    1     2     6  2027    19]
 [    0     2     2    13    46]]


In [21]:
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     26434
           1       0.99      1.00      1.00      7141
           2       1.00      1.00      1.00     38565
           3       0.96      0.99      0.97      2055
           4       0.50      0.73      0.59        63

    accuracy                           1.00     74258
   macro avg       0.89      0.94      0.91     74258
weighted avg       1.00      1.00      1.00     74258



In [22]:
print('Accuracy: ', acc)
print('Precision_weighted: ', precision)
print('Recall_weighted: ', recall)
print('mcc: ', mcc)
print('f2: ', f2)

Accuracy:  0.9967814915564653
Precision_weighted:  0.9970163134216636
Recall_weighted:  0.9967814915564653
mcc:  0.9945897601456252
f2:  0.9968284470800761
